In [11]:
import numpy as np

In [12]:
def getConserved( rho, vx, vy, P, gamma, vol ):
  """
  Calculate the conserved variable from the primitive
  rho      is matrix of cell densities
  vx       is matrix of cell x-velocity
  vy       is matrix of cell y-velocity
  P        is matrix of cell pressures
  gamma    is ideal gas gamma
  vol      is cell volume
  Mass     is matrix of mass in cells
  Momx     is matrix of x-momentum in cells
  Momy     is matrix of y-momentum in cells
  Energy   is matrix of energy in cells
  """
  Mass   = rho * vol
  Momx   = rho * vx * vol
  Momy   = rho * vy * vol
  Energy = (P/(gamma-1) + 0.5*rho*(vx**2+vy**2))*vol
  
  return Mass, Momx, Momy, Energy

In [13]:

def getPrimitive( Mass, Momx, Momy, Energy, gamma, vol ):
  """
  Calculate the primitive variable from the conservative
  Mass     is matrix of mass in cells
  Momx     is matrix of x-momentum in cells
  Momy     is matrix of y-momentum in cells
  Energy   is matrix of energy in cells
  gamma    is ideal gas gamma
  vol      is cell volume
  rho      is matrix of cell densities
  vx       is matrix of cell x-velocity
  vy       is matrix of cell y-velocity
  P        is matrix of cell pressures
  """
  rho = Mass / vol
  vx  = Momx / rho / vol
  vy  = Momy / rho / vol
  P   = (Energy/vol - 0.5*rho * (vx**2+vy**2)) * (gamma-1)
  
  return rho, vx, vy, P

In [14]:

def getGradient(f, dx):
  """
  Calculate the gradients of a field
  f        is a matrix of the field
  dx       is the cell size
  f_dx     is a matrix of derivative of f in the x-direction
  f_dy     is a matrix of derivative of f in the y-direction
  """
  # directions for np.roll() 
  R = -1   # right
  L = 1    # left
  
  f_dx = ( np.roll(f,R,axis=0) - np.roll(f,L,axis=0) ) / (2*dx)
  f_dy = ( np.roll(f,R,axis=1) - np.roll(f,L,axis=1) ) / (2*dx)
  
  return f_dx, f_dy

In [15]:
def extrapolateInSpaceToFace(f, f_dx, f_dy, dx):
  """
  Calculate the gradients of a field
  f        is a matrix of the field
  f_dx     is a matrix of the field x-derivatives
  f_dy     is a matrix of the field y-derivatives
  dx       is the cell size
  f_XL     is a matrix of spatial-extrapolated values on `left' face along x-axis 
  f_XR     is a matrix of spatial-extrapolated values on `right' face along x-axis 
  f_YR     is a matrix of spatial-extrapolated values on `left' face along y-axis 
  f_YR     is a matrix of spatial-extrapolated values on `right' face along y-axis 
  """
  # directions for np.roll() 
  R = -1   # right
  L = 1    # left
  
  f_XL = f - f_dx * dx/2
  f_XL = np.roll(f_XL,R,axis=0)
  f_XR = f + f_dx * dx/2
  
  f_YL = f - f_dy * dx/2
  f_YL = np.roll(f_YL,R,axis=1)
  f_YR = f + f_dy * dx/2
  
  return f_XL, f_XR, f_YL, f_YR

In [16]:
def getFlux(rho_L, rho_R, vx_L, vx_R, vy_L, vy_R, P_L, P_R, gamma):
  """
  Calculate fluxed between 2 states with local Lax-Friedrichs/Rusanov rule 
  rho_L        is a matrix of left-state  density
  rho_R        is a matrix of right-state density
  vx_L         is a matrix of left-state  x-velocity
  vx_R         is a matrix of right-state x-velocity
  vy_L         is a matrix of left-state  y-velocity
  vy_R         is a matrix of right-state y-velocity
  P_L          is a matrix of left-state  pressure
  P_R          is a matrix of right-state pressure
  gamma        is the ideal gas gamma
  flux_Mass    is the matrix of mass fluxes
  flux_Momx    is the matrix of x-momentum fluxes
  flux_Momy    is the matrix of y-momentum fluxes
  flux_Energy  is the matrix of energy fluxes
  """
  
  # left and right energies
  en_L = P_L/(gamma-1)+0.5*rho_L * (vx_L**2+vy_L**2)
  en_R = P_R/(gamma-1)+0.5*rho_R * (vx_R**2+vy_R**2)

  # compute star (averaged) states
  rho_star  = 0.5*(rho_L + rho_R)
  momx_star = 0.5*(rho_L * vx_L + rho_R * vx_R)
  momy_star = 0.5*(rho_L * vy_L + rho_R * vy_R)
  en_star   = 0.5*(en_L + en_R)
  
  P_star = (gamma-1)*(en_star-0.5*(momx_star**2+momy_star**2)/rho_star)
  
  # compute fluxes (local Lax-Friedrichs/Rusanov)
  flux_Mass   = momx_star
  flux_Momx   = momx_star**2/rho_star + P_star
  flux_Momy   = momx_star * momy_star/rho_star
  flux_Energy = (en_star+P_star) * momx_star/rho_star
  
  # find wavespeeds
  C_L = np.sqrt(gamma*P_L/rho_L) + np.abs(vx_L)
  C_R = np.sqrt(gamma*P_R/rho_R) + np.abs(vx_R)
  C = np.maximum( C_L, C_R )
  
  # add stabilizing diffusive term
  flux_Mass   -= C * 0.5 * (rho_L - rho_R)
  flux_Momx   -= C * 0.5 * (rho_L * vx_L - rho_R * vx_R)
  flux_Momy   -= C * 0.5 * (rho_L * vy_L - rho_R * vy_R)
  flux_Energy -= C * 0.5 * ( en_L - en_R )

  return flux_Mass, flux_Momx, flux_Momy, flux_Energy

In [17]:
def applyFluxes(F, flux_F_X, flux_F_Y, dx, dt):
  """
  Apply fluxes to conserved variables
  F        is a matrix of the conserved variable field
  flux_F_X is a matrix of the x-dir fluxes
  flux_F_Y is a matrix of the y-dir fluxes
  dx       is the cell size
  dt       is the timestep
  """
  # directions for np.roll() 
  R = -1   # right
  L = 1    # left
  
  # update solution
  F += - dt * dx * flux_F_X
  F +=   dt * dx * np.roll(flux_F_X,L,axis=0)
  F += - dt * dx * flux_F_Y
  F +=   dt * dx * np.roll(flux_F_Y,L,axis=1)
  
  return F

In [23]:
# Simulation Main Loop
gamma = 1.66
tEnd = 2
t = 0.001
vol = 0.1
courant_fac = 1
dx = 0.1

rho = 1.2
vx = 1
vy = 1
P = 100
Mass, Momx, Momy, Energy = getConserved( rho, vx, vy, P, gamma, vol )
while t < tEnd:
  
  # get Primitive variables
  rho, vx, vy, P = getPrimitive( Mass, Momx, Momy, Energy, gamma, vol )
  
  # get time step (CFL) = dx / max signal speed
  dt = courant_fac * np.min( dx / (np.sqrt( gamma*P/rho ) + np.sqrt(vx**2+vy**2)) )
  
  # calculate gradients
  rho_dx, rho_dy = getGradient(rho, dx)
  vx_dx,  vx_dy  = getGradient(vx,  dx)
  vy_dx,  vy_dy  = getGradient(vy,  dx)
  P_dx,   P_dy   = getGradient(P,   dx)
  
  # extrapolate half-step in time
  rho_prime = rho - 0.5*dt * ( vx * rho_dx + rho * vx_dx + vy * rho_dy + rho * vy_dy)
  vx_prime  = vx  - 0.5*dt * ( vx * vx_dx + vy * vx_dy + (1/rho) * P_dx )
  vy_prime  = vy  - 0.5*dt * ( vx * vy_dx + vy * vy_dy + (1/rho) * P_dy )
  P_prime   = P   - 0.5*dt * ( gamma*P * (vx_dx + vy_dy)  + vx * P_dx + vy * P_dy )
  
  # extrapolate in space to face centers
  rho_XL, rho_XR, rho_YL, rho_YR = extrapolateInSpaceToFace(rho_prime, rho_dx, rho_dy, dx)
  vx_XL,  vx_XR,  vx_YL,  vx_YR  = extrapolateInSpaceToFace(vx_prime,  vx_dx,  vx_dy,  dx)
  vy_XL,  vy_XR,  vy_YL,  vy_YR  = extrapolateInSpaceToFace(vy_prime,  vy_dx,  vy_dy,  dx)
  P_XL,   P_XR,   P_YL,   P_YR   = extrapolateInSpaceToFace(P_prime,   P_dx,   P_dy,   dx)
  
  # compute fluxes (local Lax-Friedrichs/Rusanov)
  flux_Mass_X, flux_Momx_X, flux_Momy_X, flux_Energy_X = getFlux(rho_XL, rho_XR, vx_XL, vx_XR, vy_XL, vy_XR, P_XL, P_XR, gamma)
  flux_Mass_Y, flux_Momy_Y, flux_Momx_Y, flux_Energy_Y = getFlux(rho_YL, rho_YR, vy_YL, vy_YR, vx_YL, vx_YR, P_YL, P_YR, gamma)
  
  # update solution
  Mass   = applyFluxes(Mass, flux_Mass_X, flux_Mass_Y, dx, dt)
  Momx   = applyFluxes(Momx, flux_Momx_X, flux_Momx_Y, dx, dt)
  Momy   = applyFluxes(Momy, flux_Momy_X, flux_Momy_Y, dx, dt)
  Energy = applyFluxes(Energy, flux_Energy_X, flux_Energy_Y, dx, dt)
  
  # update time
  t += dt  

AxisError: axis 0 is out of bounds for array of dimension 0